In [ ]:
Use labelmg to tag the images (choose yolo)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd

/content


In [ ]:
!ln -s /content/drive/My\ Drive// mydrive

In [ ]:
%cd mydrive

/content/drive/My Drive


In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/pjreddie/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5910, done.
remote: Total 5910 (delta 0), reused 0 (delta 0), pack-reused 5910
Receiving objects: 100% (5910/5910), 6.33 MiB | 4.16 MiB/s, done.
Resolving deltas: 100% (3922/3922), done.
Checking out files: 100% (979/979), done.


In [ ]:
%cd darknet

/content/drive/My Drive/darknet


In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:957:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN 

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
!mkdir "/content/mydrive/yolov3"

In [ ]:
!echo "defect" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.403320 0.345628 0.083984 0.106557

['0', '0.403320', '0.345628', '0.083984', '0.106557']
0 data/obj/maksssksksss0.txt
0 0.403320 0.345628 0.083984 0.106557
0 0.438750 0.338496 0.107500 0.128319

['0', '0.438750', '0.338496', '0.107500', '0.128319']
1 data/obj/maksssksksss13.txt
0 0.438750 0.338496 0.107500 0.128319
0 0.900000 0.278761 0.045000 0.070796

['0', '0.900000', '0.278761', '0.045000', '0.070796']
1 data/obj/maksssksksss13.txt
0 0.900000 0.278761 0.045000 0.070796
0 0.755000 0.323009 0.060000 0.070796

['0', '0.755000', '0.323009', '0.060000', '0.070796']
1 data/obj/maksssksksss13.txt
0 0.755000 0.323009 0.060000 0.070796
0 0.474093 0.471250 0.367876 0.142500

['0', '0.474093', '0.471250', '0.367876', '0.142500']
2 data/obj/maksssksksss15.txt
0 0.474093 0.471250 0.367876 0.142500
0 0.510363 0.883750 0.202073 0.082500

['0', '0.510363', '0.883750', '0.202073', '0.082500']
2 data/obj/maksssksksss15.txt
0 0.510363 0.883750 0.202073 0.082500
0 0.218750 0.201724 0.097500 0.0724

In [ ]:
import glob
images_list = glob.glob("data/obj/*.png")
len(images_list)

11

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
!chmod 755 darknet

In [ ]:
!apt install gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc 
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  cpp-5 gcc-5-base libasan2 libgcc-5-dev libisl15 libmpx0 libstdc++-5-dev
Suggested packages:
  gcc-5-locales g++-5-multilib gcc-5-doc libstdc++6-5-dbg gcc-5-multilib
  libgcc1-dbg libgomp1-dbg libitm1-dbg libatomic1-dbg libasan2-dbg
  liblsan0-dbg libtsan0-dbg libubsan0-dbg libcilkrts5-dbg libmpx0-dbg
  libquadmath0-dbg libstdc++-5-doc
The following NEW packages will be installed:
  cpp-5 g++-5 gcc-5 gcc-5-base libasan2 libgcc-5-dev libisl15 libmpx0
  libstdc++-5-dev
0 upgraded, 9 newly installed, 0 to remove and 35 not upgraded.
Need to get 29.1 MB of archives.
After this operation, 100 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 gcc-5-base amd64 

In [ ]:
!make

make: Nothing to be done for 'all'.


In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show


Streaming output truncated to the last 5000 lines.
Region 82 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.397062, .5R: -nan, .75R: -nan,  count: 0
Region 94 Avg IOU: 0.275427, Class: 0.607140, Obj: 0.338275, No Obj: 0.467183, .5R: 0.166667, .75R: 0.000000,  count: 6
Region 106 Avg IOU: 0.128242, Class: 0.220220, Obj: 0.073787, No Obj: 0.370782, .5R: 0.000000, .75R: 0.000000,  count: 2
Region 82 Avg IOU: 0.178636, Class: 0.492549, Obj: 0.638279, No Obj: 0.395778, .5R: 0.000000, .75R: 0.000000,  count: 1
Region 94 Avg IOU: 0.376652, Class: 0.481434, Obj: 0.503795, No Obj: 0.465592, .5R: 0.333333, .75R: 0.000000,  count: 3
Region 106 Avg IOU: 0.156499, Class: 0.494818, Obj: 0.346076, No Obj: 0.374177, .5R: 0.000000, .75R: 0.000000,  count: 3
Region 82 Avg IOU: 0.188396, Class: 0.797310, Obj: 0.567959, No Obj: 0.398333, .5R: 0.250000, .75R: 0.000000,  count: 4
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.466581, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, C